In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML, clear_output
display(HTML('<style>.container { width:80% !important; }</style>'))
display(HTML('<style>.prompt { min-width:10ex !important; }</style>'))
display(HTML('<style>div#notebook { font-size:12px !important; }</style>'))

from preprocessing import leave_last_out, transform_indices, preprocessing,\
reindex_data, generate_interactions_matrix, get_interaction_matrix
from datetime import datetime
from scipy.sparse import csr_matrix
from preproc_deep_fm import load_csv
from src.baselines.LightFM.Lightfm import build_lfm_model, lightfm_scoring
from lightfm import LightFM
import pickle
from evaluation import topn_recommendations

C:\Users\Максимилиан\AppData\Local\Temp\ipykernel_10972\1525839755.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output
C:\Users\Максимилиан\AppData\Local\Temp\ipykernel_10972\1525839755.py:3: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output


C:\Users\Максимилиан\AppData\Roaming\Python\Python39\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
user_df = pd.read_csv('./dataset/users_processed.csv')
item_df = pd.read_csv('./dataset/items_processed.csv')
interaction_df = pd.read_csv('./dataset/interactions_processed.csv', parse_dates=['last_watch_dt'])
#submission = pd.read_csv('sample_submission.csv')

In [3]:
def ohe(features : list, df, items = False):
    if items:
        ohe_df = df.item_id
    else:
        ohe_df = df.user_id
    for feat in features:
        ohe_feat_df = pd.get_dummies(df[feat], prefix = feat)
        ohe_df = pd.concat([ohe_df,ohe_feat_df],axis=1)
    return ohe_df

In [4]:
# One-hot encoding of cathegorical features
user_cat_features = ['age','income','sex','kids_flg']

user_ohe_df  =  ohe(user_cat_features,user_df) #.drop(columns='user_id')

item_cat_feats = ['content_type','release_year_cat','for_kids','age_rating','studios','countries','directors']
item_ohe_df = ohe(item_cat_feats,item_df, items = True) #.drop(columns='item_id')

In [5]:
train_val, data_description, train_matrix, train_matrix_indices, cold_users_matrix, cold_start_matrix_indices = \
preprocessing(interaction_df,user_ohe_df,item_ohe_df)

C:\Users\Максимилиан\Desktop\Skoltech\Intro to RecSys\FP\preprocessing.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions_df['last_watch_dt_ts'] = interactions_df['last_watch_dt'].apply(lambda x: int(x.timestamp()))
C:\Users\Максимилиан\Desktop\Skoltech\Intro to RecSys\FP\preprocessing.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['uid'] = data[userid].astype('category')
C:\Users\Максимилиан\Desktop\Skoltech\Intro to RecSys\FP\preprocessing.py:208: SettingWithCopyWarning: 
A value

In [6]:
lfm_config = dict(
    no_components = 50,
    loss = 'warp',
    max_sampled = 3,
    max_epochs = 100,
    learning_schedule = 'adagrad',
    user_alpha = 1e-3,
    item_alpha = 1e-3,
)


In [7]:
lfm = LightFM(
        no_components=lfm_config['no_components'],
        loss=lfm_config['loss'],
        learning_schedule=lfm_config['learning_schedule'],
        # learning_rate=
        user_alpha=lfm_config['user_alpha'],
        item_alpha=lfm_config['item_alpha'],
        max_sampled=lfm_config['max_sampled'],
        # random_state =
    )

In [43]:
lfm.fit(train_matrix,
        user_features=data_description['user_features'],
        item_features=data_description['item_features'],
        epochs=20,
        verbose = True)

Epoch: 100%|██████████████████████████████████████████████████████████████████████████| 20/20 [34:55<00:00, 104.78s/it]


In [24]:
with open('LFMmodel.pkl', 'wb') as f:
    pickle.dump(lfm, f)

In [17]:
with open('LFMmodel.pkl', 'rb') as f:
    lfm = pickle.load(f)

In [18]:
holdout = data_description['holdout_standard'][data_description['holdout_standard'].user_id.isin(train_val.user_id)]

In [19]:
def user_evaluate(user_id, preds, holdout,data_description):
        n_items = data_description['n_items']
        holdout_user = holdout[holdout.user_id == user_id]
        predictions = preds[user_id]
        hits_mask = predictions == holdout_user.item_id.values
        # HR calculation
        hr = np.mean(hits_mask.any(axis=0))
        # MRR calculation
        n_test_users = preds.shape[0]
        hit_rank = np.where(hits_mask)[0] + 1.0

        mrr = np.sum(1 / hit_rank)
        # coverage calculation
        cov = np.unique(preds).size / n_items
        # ndcg
        # NDCG
        ndcg_per_user = 1.0 / np.log2(hit_rank + 1)
        ndcg = np.sum(ndcg_per_user) / n_test_users

        return hr, mrr, cov, ndcg

In [20]:
topn = 20

In [21]:
dtype = 'i4'
all_users = np.arange(data_description['n_users_train'], dtype=dtype)
test_items = np.arange(data_description['n_items']).astype(dtype)
item_index, user_index = np.meshgrid(test_items, all_users, copy=False)
all_items = np.arange(data_description['n_items'], dtype=dtype)

In [22]:
def lightfm_scoring(lfm, preds, data_description, users_to_val = 10000):
    dtype = 'i4'
    all_users = np.arange(data_description['n_users_train'], dtype=dtype)
    test_items = np.arange(data_description['n_items']).astype(dtype)
    item_index, user_index = np.meshgrid(test_items, all_users, copy=False)
    all_items = np.arange(data_description['n_items'], dtype=dtype)
    
    
    for i in range(len(all_users)):
        if i % 1000 == 0:
            print(i)
        if i == users_to_val:
            break
        score = lfm.predict(user_index[i].ravel(),item_ids = all_items.ravel(), 
                 user_features=data_description['user_features'],
                 item_features=data_description['item_features'],
                 num_threads = 4)
        scores = np.expand_dims(score,axis=0)
        scores_topn = topn_recommendations(scores,topn)
        preds[i,:] = scores_topn
    
    
    hr_full = []
    mrr_full = []
    cov_full = []
    ndcg_full = []
    for i in holdout.user_id.sort_values():
        if i % 1000 == 0:
            print(i)
        if i == users_to_val:
            break
        predictions = preds[i]

        hr, mrr, cov, ndcg = user_evaluate(i,preds, holdout)
        hr_full.append(hr)
        mrr_full.append(mrr)
        cov_full.append(cov)
        ndcg_full.append(ndcg)
        
        
        
    return np.array(hr_full), np.array(mrr_full), np.array(cov_full)
    
    
    

In [23]:
preds = np.zeros((all_users.shape[0],topn))

for i in range(len(all_users)):
    if i % 1000 == 0:
        print(i)
    if i == 10000:
        break
    score = lfm.predict(user_index[i].ravel(),item_ids = all_items.ravel(), 
             user_features=data_description['user_features'],
             item_features=data_description['item_features'],
             num_threads = 4)
    scores = np.expand_dims(score,axis=0)
    scores_topn = topn_recommendations(scores,topn)
    preds[i,:] = scores_topn

0
1000



KeyboardInterrupt



In [38]:
preds[4]

array([14633., 13283.,  9307.,  3748., 13639.,  3337.,  8684.,  2181.,
        6753.,   128., 12367., 10023.,  4401., 10869.,   752., 10852.,
       12871.,  3712.,  3872.,   241.])

In [39]:
holdout[holdout['user_id'] == 4]

,user_id,item_id,last_watch_dt,total_dur,watched_pct,last_watch_dt_ts
114632,4,4204,2021-07-02,5799,97,1625184000


In [11]:
scores_topn

NameError: name 'scores_topn' is not defined

In [29]:
def lightfm_eval(preds,holdout,data_description, users_to_eval = 10000):

    hr_full = []
    mrr_full = []
    cov_full = []
    ndcg_full = []
    for count,i in enumerate(holdout.user_id.sort_values()):
        if count % 100 == 0:
            print(count,i)
        if count == users_to_eval:
            break
       # predictions = preds[i]

        hr, mrr, cov, ndcg = user_evaluate(i,preds, holdout, data_description)
        hr_full.append(hr)
        mrr_full.append(mrr)
        cov_full.append(cov)
        ndcg_full.append(ndcg)
    return np.array(hr_full), np.array(mrr_full), np.array(cov_full),  np.array(ndcg_full)

In [20]:
hr = np.array(hr_full)
mrr = np.array(mrr_full)
cov = np.array(cov_full) 
ndcg = np.array(ndcg_full) 

In [21]:
print('HR mean:',hr.mean())
print('MRR mean:',mrr.mean())
print('Coverage mean:',cov.mean())
print('NDCG mean:',ndcg.mean())

HR mean: 0.26378725682669996
MRR mean: 0.04384547094028249
Coverage mean: 0.0017661843624753753
NDCG mean: 2.553060354700773e-07


# Coldstart

In [13]:
def cold_start_preds(model,user_matrix):
    item_feature_bias, item_feature_factors = model.get_item_representations()
    item_factors_inv = np.linalg.pinv(item_feature_factors[:,:])
    user_feature_bias, user_feature_factors = model.get_user_representations()
    
    user_factors = (item_factors_inv @ user_matrix.T).T
    predictions = user_factors @ item_factors.T + item_bias
  
    return predictions

In [14]:
item_feature_bias, item_feature_factors = lfm.get_item_representations()
item_factors_inv = np.linalg.pinv(item_feature_factors[:,:])
user_feature_bias, user_feature_factors = lfm.get_user_representations()

In [15]:
item_features = data_description['item_features']

In [16]:
item_factors = (item_features @ item_feature_factors)
item_bias = (item_features @ item_feature_bias)
item_factors_inv = np.linalg.pinv(item_factors)

In [9]:
data_description

{'users': 'user_id',
 'items': 'item_id',
 'feedback': 'watched_pct',
 'n_users_train': 362088,
 'n_items': 14721,
 'user_features': <362088x19 sparse matrix of type '<class 'numpy.uint8'>'
 	with 1448352 stored elements in Compressed Sparse Row format>,
 'item_features': <14721x8588 sparse matrix of type '<class 'numpy.uint8'>'
 	with 103047 stored elements in Compressed Sparse Row format>,
 'holdout_standard':          user_id  item_id last_watch_dt  total_dur  watched_pct  \
 1807000   372961     4379    2021-03-13         58            1   
 4022024   542099     2757    2021-03-13       6496          100   
 4777864   881803    13463    2021-03-13         75            1   
 5224013   280001     1215    2021-03-13        246            4   
 745212    936870       23    2021-03-13      11604          100   
 ...          ...      ...           ...        ...          ...   
 240772    642282     9166    2021-08-22       1044           17   
 1022745   829290      993    2021-08-22 

In [18]:
cu_user = cold_users_matrix[:20000,:]

In [19]:
user_factors = (item_factors_inv @ cu_user.T).T
predictions = user_factors @ item_factors.T + item_bias

In [20]:
holdout_cs = data_description['holdout_cs']

In [21]:
scores_topn = topn_recommendations(predictions,topn)

In [30]:
hr, mrr, cov, ndcg = lightfm_eval(scores_topn,holdout_cs,data_description, users_to_eval = 2000)

0 10
100 894
200 1863
300 2821
400 3804
500 4589
600 5377
700 6214
800 7124
900 8061
1000 9035
1100 9834
1200 10814
1300 11591
1400 12357
1500 13231
1600 14033
1700 14901
1800 15760
1900 16746
2000 17627


In [31]:
print('HR mean:',hr.mean())
print('MRR mean:',mrr.mean())
print('Coverage mean:',cov.mean())
print('NDCG mean:',ndcg.mean())

HR mean: 0.002
MRR mean: 0.00088125
Coverage mean: 0.1665647714149854
NDCG mean: 5.656082757310562e-08
